<a href="https://colab.research.google.com/github/yustiks/photolab_2/blob/master/image_mixing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from __future__ import print_function
from matplotlib import pyplot as plt
import cv2
import numpy as np
import glob
from cv2 import VideoWriter, VideoWriter_fourcc
from datetime import datetime
from PIL import Image, ImageFont, ImageDraw


def drawTextWithOutline(text, x, y, font):

    draw.text((x-2, y-2), text,(0,0,0),font=font)
    draw.text((x+2, y-2), text,(0,0,0),font=font)
    draw.text((x+2, y+2), text,(0,0,0),font=font)
    draw.text((x-2, y+2), text,(0,0,0),font=font)
    draw.text((x, y), text, (255,255,255), font=font)
    return
  
def drawText(text, pos, font):
    text = text.upper()

    w, h = draw.textsize(text, font) # measure the size the text will take

    lineCount = 1
    if w > img.width:
        lineCount = int(round((w / img.width) + 1))

    lines = []

    if lineCount > 1:

        lastCut = 0
        isLast = False
        for i in range(0,lineCount):
            if lastCut == 0:
                cut = int(len(text) / lineCount) * i
            else:
                cut = lastCut

            if i < lineCount-1:
                nextCut = int(len(text) / lineCount) * (i+1)
            else:
                nextCut = len(text)
                isLast = True

            # make sure we don't cut words in half
            if nextCut == len(text) or text[nextCut] == " ":
                pass
            else:
                while text[nextCut] != " ":
                    nextCut += 1

            line = text[cut:nextCut].strip()

            # is line still fitting ?
            w, h = draw.textsize(line, font)
            if not isLast and w > img.width:
                nextCut -= 1
                while text[nextCut] != " ":
                    nextCut -= 1

            lastCut = nextCut
            lines.append(text[cut:nextCut].strip())

    else:
        lines.append(text)


    lastY = -h
    if pos == "bottom":
        lastY = img.height - h * (lineCount+1) - 10
    return lineCount, lines, lastY
  
def draw_all(lineCount, lines, lastY, font):
  for i in range(0, lineCount):
      w, h = draw.textsize(lines[i], font)
      x = img.width/2 - w/2
      y = lastY + h
      drawTextWithOutline(lines[i], x, y, font)
      lastY = y
        
        
t = datetime.now()

# [load]
src1 = cv2.imread('alena.png')
src2 = cv2.imread('tree.jpg')

height, width, depth = src2.shape
imgScale = 700/width
newX,newY = src1.shape[1]*imgScale, src1.shape[0]*imgScale
size = (int(newX),int(newY))
src1 = cv2.resize(src1, size)
img = Image.fromarray(src1)
gray = cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

src2 = cv2.resize(src2,(int(newX),int(newY)))

video = cv2.VideoWriter('project.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

text = "Юстина make memes great again with several lines make memes great again with several lines"
if len(text)<10:
  font = ImageFont.truetype("impact.ttf", int(72*imgScale))
elif len(text)<40:
  font = ImageFont.truetype("impact.ttf", int(62*imgScale))
elif len(text)<60:
  font = ImageFont.truetype("impact.ttf", int(52*imgScale))
else:
  font = ImageFont.truetype("impact.ttf", int(42*imgScale))
lineCount, lines, lastY = drawText(text, 'bottom', font)

for i in range(0,75): 
	# [blend_images]
	alpha = i/75
	beta = (1.0 - alpha)
	dst = cv2.addWeighted(gray, alpha, src1, beta, 0.0)
	img = Image.fromarray(dst)
	draw = ImageDraw.Draw(img)
	draw_all(lineCount, lines, lastY, font)
#	drawText(text, "bottom", font)
	video.write(np.array(img))
for i in range(75,400): 
	# [blend_images]
	alpha = (i - 75)/325
	beta = (1.0 - alpha)
	dst = cv2.addWeighted(src2, alpha, gray, beta, 0.0)
	img = Image.fromarray(dst)
	draw = ImageDraw.Draw(img)
	draw_all(lineCount, lines, lastY, font)
#	drawText(text, "bottom", font)
	img.save("out.jpg")
	video.write(np.array(img))
	if i == 300:
		break
print(datetime.now()-t)
video.release()	


0:00:03.813235


In [36]:
from moviepy.editor import *
t = datetime.now()
videoclip = VideoFileClip("project.mp4")
videoclip.write_videofile("output.mp4", audio="hello_1.mp3")
print(datetime.now()-t)

[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|██████████| 301/301 [00:01<00:00, 189.75it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

0:00:01.822269
